# Distil-Whisper

Distil-Whisper is a distilled version of the Whisper model that is 6 times faster, 49% smaller, and performs within 1% WER on out-of-distribution evaluation sets.

Before getting started, let's quickly recap Whisper. Whisper is a general purpose speech recognition model proposed by OpenAI in the paper [*Robust Speech Recognition via Large-Scale Weak Supervision*](https://cdn.openai.com/papers/whisper.pdf). The Whisper architecture is a Transformer based encoder-decoder model. First, the encoder maps the input audio to encoder hidden-states in a single forward pass. The decoder then auto-regressively predicts text tokens, conditional on both the previous tokens and the encoder hidden-states.

OpenAI's best Whisper checkpoint, named [Whisper large-v2](https://huggingface.co/openai/whisper-large-v2), has 32 encoder layers and 32 decoder layers. 32 layers is quite a lot! Let's visualize the model:

![img](https://huggingface.co/datasets/patrickvonplaten/images_distil/resolve/main/whiser_arch_old.png)

Here, $\mathbf{X}_{1:T}$ represents the speech input. It is mapped by the encoder (shown in green) through a single forward pass. The encoder ouputs, i.e. the  encoder hidden-states $\mathbf{H}_{1:M}$, are then used in the cross-attention layers in each decoder block.

Starting with a start-of-sequence token $y_0$, the decoder (shown in orange) auto-regressively generates the text tokens in the transcription. In the visiualization above, there are 5 decoder forward passes, one for each $\mathbf{P}(y_i | \mathbf{y}_{0: i - 1}),  \forall i$.

In practice, the decoder is run up to 128 times (depending on the length of the transcription), which means that there many more forward passes through the decoder then the encoder. The result is that the decoder is responsible for over **90% of the inference time** in Whisper.

This is the motivation behind Distil-Whipser: we make the decoder faster in order to speed-up the inference time of model. With this in mind, let's take a look at the Distil-Whisper architecture:

![img](https://huggingface.co/datasets/patrickvonplaten/images_distil/resolve/main/distil_arch_old.png)

Just two decoder layers! That means to generate a transcription of 128 tokens, Distil-Whisper needs to run only 256 decoder layer forward passes, while Whisper large-v2 has to run 4096 forward passes. Since the encoder is only run once, we copy the entire encoder and *freeze* it during training. This means Distil-Whisper inherits Whisper's robustness to different audio conditions.

## Benchmarking

Great, now that we've understood why Distil-Whisper should be faster in theory, let's see if it holds true in practice.

To begin with, we install `transformers`, `accelerate`, and `datasets`.

In this notebook, we use a A100 GPU that is available through a Colab pro subscription, as this is the device we used for benchmarking in the [Distil-Whisper paper](https://huggingface.co/papers/2311.00430). Other GPUs will most likely lead to different speed-ups, but they should be in the same ballpark range:

In [2]:
%pip install wheel python-dotenv torch tensor jupyter ipywidgets
%pip install --upgrade transformers accelerate datasets datasets\[audio\]
%pip install flash-attn --no-build-isolation

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In addition, we will make use of [Flash Attention 2](https://github.com/Dao-AILab/flash-attention), as it saves
a lot of memory and speeds up large matmul operations.

In [4]:
# Load environment variables
import os
from dotenv import load_dotenv

load_dotenv('../hf.env')
HF_TOKEN_PATH = os.getenv("HF_TOKEN_PATH")
HF_USERNAME = os.getenv("HF_USERNAME")

# Read the token if it is available
try:
    with open(HF_TOKEN_PATH, 'r') as token_file:
        HF_TOKEN = token_file.read().strip()
except FileNotFoundError as e:
    print(f"Error: {e}")

In [5]:
# Authenticate with Hugging Face Hub
from huggingface_hub import login

# Check if the token is set
if HF_TOKEN:
    # Use the token to log in
    login(token=HF_TOKEN)
else:
    login()

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /media/bigdaddy/data/cache_huggingface/token
Login successful


To begin with, let's load the dataset that we will use for benchmarking. We'll load a small dataset consisting of 73 samples from the [LibriSpeech ASR](https://huggingface.co/datasets/librispeech_asr) validation-clean dataset. This amounts to ~9MB of data, so it's very lightweight and quick to download on device:

In [6]:
from datasets import load_dataset

dataset = load_dataset("hf-internal-testing/librispeech_asr_dummy", "clean", split="validation", trust_remote_code=True)

We start by benchmarking [Whisper large-v2](https://huggingface.co/openai/whisper-large-v2) to get our baseline number. We'll load the model in `float16` precision and make sure that loading time takes as little time as possible by passing `low_cpu_mem_usage=True`. In addition, we want to make sure that the model is loaded in [`safetensors`](https://github.com/huggingface/safetensors) format by passing `use_safetensors=True`:

In [7]:
from transformers import AutoModelForSpeechSeq2Seq, AutoProcessor
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32

print(f'device: {device}')
print(f'torch_dtype: {torch_dtype}')

model_id = "openai/whisper-large-v2"

model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id,
    torch_dtype=torch_dtype,
    low_cpu_mem_usage=True,
    use_safetensors=True,
    attn_implementation="flash_attention_2"
)

print(f'model device pre: {model.device}')
model.to(device)
print(f'model device post: {model.device}')

processor = AutoProcessor.from_pretrained(model_id)

/home/bigdaddy/Documents/GitHub/ucsd-ml-ai-capstone/reproduce-existing-solutions/Distil-Whisper/.venv/lib/python3.10/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


device: cuda
torch_dtype: torch.float16


You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.


model device pre: cpu
model device post: cuda:0


Great! For the benchmark, we will only measure the generation time (encoder + decoder), so let's write a short helper function that measures this step:

In [8]:
import time

def generate_with_time(model, inputs):
    start_time = time.time()
    outputs = model.generate(**inputs)
    generation_time = time.time() - start_time
    return outputs, generation_time

This function will return both the decoded tokens as well as the time
it took to run the model.

We now iterate over the audio samples and sum up the generation time.

In [9]:
from tqdm import tqdm

all_time = 0

for sample in tqdm(dataset):
  audio = sample["audio"]
  inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt")
  inputs = inputs.to(device=device, dtype=torch.float16)

  output, gen_time = generate_with_time(model, inputs)
  all_time += gen_time
  print(processor.batch_decode(output, skip_special_tokens=True))

print(all_time)

  0%|          | 0/73 [00:00<?, ?it/s]Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
  1%|▏         | 1/73 [00:01<01:40,  1.39s/it]

[' Mr. Quilter is the apostle of the middle classes and we are glad to welcome his gospel.']


  3%|▎         | 2/73 [00:01<00:51,  1.38it/s]

[" Nor is Mr. Quilter's manner less interesting than his matter."]


  4%|▍         | 3/73 [00:02<00:42,  1.66it/s]

...

[" Breon saw something close to panic on his opponent's face when the man finally recognized his error."]


 99%|█████████▊| 72/73 [00:22<00:00,  3.27it/s]

[' A wave of despair rolled out from Irolg. Brienne sensed it and knew the fifth point was his.']


100%|██████████| 73/73 [00:23<00:00,  3.16it/s]

[" Then the powerful twist that's rested aside, in and under the guard."]
21.717344999313354


Alright! In total it took roughly 63 seconds to transcribe 73 audio samples.

Next, let's see how much time it takes with [Distil-Whisper](https://huggingface.co/distil-whisper/distil-large-v2):

In [10]:
model_id = "distil-whisper/distil-large-v2"

distil_model = AutoModelForSpeechSeq2Seq.from_pretrained(
    model_id, torch_dtype=torch_dtype, low_cpu_mem_usage=True, use_safetensors=True, attn_implementation="flash_attention_2"
)
distil_model = distil_model.to(device)

We run the same benchmarking loop:

In [11]:
all_time = 0

for sample in tqdm(dataset):
  audio = sample["audio"]
  inputs = processor(audio["array"], sampling_rate=audio["sampling_rate"], return_tensors="pt")
  inputs = inputs.to(device=device, dtype=torch.float16)

  output, gen_time = generate_with_time(distil_model, inputs)
  all_time += gen_time
  print(processor.batch_decode(output, skip_special_tokens=True))

print(all_time)

  3%|▎         | 2/73 [00:00<00:05, 12.02it/s]

[' Mr. Quilter is the apostle of the middle classes, and we are glad to welcome his gospel.']
[" Nor is Mr. Quilter's manner less interesting than his matter."]
[' He tells us that at this festive season of the year, with Christmas and roast beef looming before us, similes drawn from eating and its results occur most readily to the mind.']


  5%|▌         | 4/73 [00:00<00:06, 10.01it/s]

[" He has grave doubts whether Sir Frederick Leighton's work is really Greek after all, and can discover in it but little of Rocky Ithaca."]


  8%|▊         | 6/73 [00:00<00:08,  8.14it/s]

...

[' Others had died before during the 20s, and death during the last round was, in some ways, easier than defeat.']
[' Breathing deeply, Briann softly spoke the autohypnotic phrases that triggered the process.']
[' When the buzzer sounded, he pulled his foil from his second startled grasp and ran forward.']


 96%|█████████▌| 70/73 [00:06<00:00, 11.97it/s]

[' I rolled to look amazed at the sudden fury of the attack, then smiled.']
[' He thought it was the last burst of energy. He knew how close they both were to exhaustion.']
[" Brian saw something close to panic on his opponent's face when the man finally recognized his error."]


100%|██████████| 73/73 [00:06<00:00, 11.65it/s]

[' A wave of despair rolled out from Irog. Brian sensed it and knew the fifth point was his.']
[" Then the powerful twist that's rested aside, in and under the guard."]
5.49275803565979


Only 10 seconds - that amounts to a 6x speed-up!

## Memory

In addition to being significantly faster, Distil-Whisper also has fewer parameters. Let's have a look at how many fewer exactly.

In [12]:
distil_model.num_parameters() / model.num_parameters() * 100

49.000047275167184

Distil-Whisper is 49% of the size of Whisper. Note that this ratio is much lower if we would just compare the size of the decoder:

In [13]:
distil_model.model.decoder.num_parameters() / model.model.decoder.num_parameters() * 100


13.175161920253482

As expected the decoder is much smaller. One might have guessed that it should even be less, around 2/32 (or 6%), but we can't forget that the decoder has a very large word embedding that requires a lot of parameters.

## Next steps

Hopefully this notebook shed some light on the motivation behind Distil-Whisper! For now, we've measured Distil-Whisper mainly on GPU, but are now actively looking into collaborating to release code how to effectively accelerate Distil-Whisper on CPU as well. Updates will be posted on the Distil-Whisper [repository](https://github.com/huggingface/distil-whisper).

Another key application of Distil-Whisper is *speculative decoding*. In speculative decoding, we can use Distil-Whisper as an *assitant model* to Whisper-large-v2 to reach a speed-up of 2x without **any** loss in performance. More on that in a follow-up notebook that will come out soon!